<a href="https://www.kaggle.com/code/adityamishra2612/qlub-australia-assignment?scriptVersionId=251159070" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/training/Seasonality Model.xlsx


**Data loading and detailed overview**

> in this step, I have loaded the dataset and had a detailed overview, which includes the dataset's shape, column names, sample rows, data types, and basic statistical summaries.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import seaborn as sns

df = pd.read_excel('/kaggle/input/training/Seasonality Model.xlsx')

print("Dataset Shape:", df.shape)
print("\nColumn Names:")
print(df.columns.tolist())
print("\nFirst few rows:")
print(df.head())
print("\nData Types:")
print(df.dtypes)
print("\nBasic Statistics:")
print(df.describe())

Dataset Shape: (24875, 5)

Column Names:
['Restaurant ID', 'Go Live Date', 'Date Level [Finance]', 'GMV', 'Revenue']

First few rows:
            Restaurant ID Go Live Date Date Level [Finance]      GMV  Revenue
0  Little-Venice-Pizzeria   2023-01-05           2023-01-05    40.78     0.88
1          Sujan-Silpakar   2023-01-12           2023-01-12    10.25     0.00
2                    Poly   2023-01-17           2023-01-17   959.82    11.18
3                    Poly   2023-01-17           2023-01-18   939.26    12.05
4                    Poly   2023-01-17           2023-01-19  1463.39    18.79

Data Types:
Restaurant ID                   object
Go Live Date            datetime64[ns]
Date Level [Finance]    datetime64[ns]
GMV                            float64
Revenue                        float64
dtype: object

Basic Statistics:
                        Go Live Date           Date Level [Finance]  \
count                          24875                          24875   
mean   2023-11-

**Here, the analysis aims to understand the dataset by examining the number of unique restaurants, their date ranges, missing values, data completeness, and the distribution of key financial metrics, which provides a comprehensive overview of the data quality and structure.**

In [3]:
print("Number of unique restaurants:", df['Restaurant ID'].nunique())
print("\nUnique restaurants:")
unique_restaurants = df['Restaurant ID'].unique()
print(unique_restaurants[:20])  # Show first 20

print("\nDate range:")
print("Start date:", df['Date Level [Finance]'].min())
print("End date:", df['Date Level [Finance]'].max())

print("\nMissing values:")
print(df.isnull().sum())

restaurant_counts = df['Restaurant ID'].value_counts()
print("\nTop 10 restaurants by number of records:")
print(restaurant_counts.head(10))

print("\nGMV and Revenue distribution:")
print(df[['GMV', 'Revenue']].describe())

Number of unique restaurants: 233

Unique restaurants:
['Little-Venice-Pizzeria' 'Sujan-Silpakar' 'Poly' 'Bella-Sydney'
 'saga-bar-sydney' 'The-Good-Gypsy' 'Nikkei-Bar-Sydney' 'Stray-Neighbour'
 'Salona-Melbourne' 'Da-Orazio' 'James-South-Melbourne'
 'Rococo-Acland-Street' 'Emerge-Dining-Geelong' 'Entrata-restaurant'
 'Tinto-Hawthorn' 'Fitzrovia-St-Kilda' 'Bang-Bang-Mordialloc'
 'Beachcomber-StKilda' 'Imaccheroni-catering' 'thirty-eight-chairs']

Date range:
Start date: 2023-01-05 00:00:00
End date: 2024-12-31 00:00:00

Missing values:
Restaurant ID            0
Go Live Date             0
Date Level [Finance]     0
GMV                     23
Revenue                 23
dtype: int64

Top 10 restaurants by number of records:
Restaurant ID
saga-bar-sydney                 641
Mercetta-Mornington             563
Daydreamers-Cafe                544
Martini-and-Co-Plenty-Valley    531
iceberg-bondi                   508
Onda-Bar-Melbourne              506
Cosi-Restaurant-Melbourne       436
Va

**The data is being cleaned by removing missing values and converting date columns to datetime format to ensure accurate time-based analysis. It then extracts monthly and yearly information to facilitate seasonality analysis and aggregates revenue and GMV data monthly per restaurant, providing a summarised view of the data's structure and coverage.**

In [4]:
df_clean = df.dropna(subset=['GMV', 'Revenue']).copy()


df_clean['Date Level [Finance]'] = pd.to_datetime(df_clean['Date Level [Finance]'])
df_clean['Go Live Date'] = pd.to_datetime(df_clean['Go Live Date'])

df_clean['Month'] = df_clean['Date Level [Finance]'].dt.month
df_clean['Year'] = df_clean['Date Level [Finance]'].dt.year
df_clean['YearMonth'] = df_clean['Date Level [Finance]'].dt.to_period('M')

print("Cleaned dataset shape:", df_clean.shape)
print("Date range after cleaning:")
print("Start:", df_clean['Date Level [Finance]'].min())
print("End:", df_clean['Date Level [Finance]'].max())

monthly_data = df_clean.groupby(['Restaurant ID', 'YearMonth']).agg({
    'Revenue': ['sum', 'count', 'mean'],
    'GMV': ['sum', 'mean']
}).reset_index()

monthly_data.columns = ['Restaurant ID', 'YearMonth', 'Revenue_Sum', 'Revenue_Count', 'Revenue_Mean', 'GMV_Sum', 'GMV_Mean']

print("\nMonthly aggregated data shape:", monthly_data.shape)
print("\nSample of monthly data:")
print(monthly_data.head())

months_per_restaurant = monthly_data.groupby('Restaurant ID')['YearMonth'].nunique().sort_values(ascending=False)
print("\nTop 10 restaurants by number of months of data:")
print(months_per_restaurant.head(10))

Cleaned dataset shape: (24852, 8)
Date range after cleaning:
Start: 2023-01-05 00:00:00
End: 2024-12-31 00:00:00

Monthly aggregated data shape: (1388, 7)

Sample of monthly data:
          Restaurant ID YearMonth  Revenue_Sum  Revenue_Count  Revenue_Mean  \
0          1800-Lasagne   2023-06      1193.73             14     85.266429   
1   ATOS-restaurant-QLD   2024-09         7.02              2      3.510000   
2   ATOS-restaurant-QLD   2024-10        33.76              1     33.760000   
3  Aces-Gouger-Adelaide   2024-10      1278.70             21     60.890476   
4  Aces-Gouger-Adelaide   2024-11       261.63              2    130.815000   

    GMV_Sum     GMV_Mean  
0  57995.32  4142.522857  
1    392.22   196.110000  
2   1951.96  1951.960000  
3  70776.98  3370.332381  
4  14037.79  7018.895000  

Top 10 restaurants by number of months of data:
Restaurant ID
saga-bar-sydney                 23
Entrata-restaurant              21
Onda-Bar-Melbourne              20
Imaccheroni-cat

**The reasoning is to analyse restaurant revenue data over at least six months to determine seasonality patterns by calculating indices and variability measures for each qualified restaurant, enabling the identification of seasonal trends in revenue.**

In [5]:
min_months_threshold = 6

restaurant_months = monthly_data.groupby('Restaurant ID')['YearMonth'].nunique()
qualified_restaurants = restaurant_months[restaurant_months >= min_months_threshold].index

print(f"Number of restaurants with at least {min_months_threshold} months of data: {len(qualified_restaurants)}")

qualified_data = monthly_data[monthly_data['Restaurant ID'].isin(qualified_restaurants)].copy()

seasonality_results = []

for restaurant in qualified_restaurants:
    restaurant_data = qualified_data[qualified_data['Restaurant ID'] == restaurant].copy()
    

    if restaurant_data['Revenue_Sum'].sum() == 0:
        continue
    
    monthly_avg = restaurant_data.groupby(restaurant_data['YearMonth'].dt.month)['Revenue_Sum'].mean()
    
    mean_revenue = restaurant_data['Revenue_Sum'].mean()
    std_revenue = restaurant_data['Revenue_Sum'].std()
    
    seasonality_indices = monthly_avg / mean_revenue if mean_revenue != 0 else np.nan
    
    cv = std_revenue / mean_revenue if mean_revenue != 0 else np.nan
    
    result = {
        'Restaurant_ID': restaurant,
        'Overall_Monthly_Avg': mean_revenue,
        'CV': cv,
        'Months_of_Data': len(restaurant_data)
    }
    
    for month in range(1, 12 + 1):
        result[f'Month_{month}_Index'] = seasonality_indices.get(month, np.nan)
    
    seasonality_results.append(result)


seasonality_df = pd.DataFrame(seasonality_results)

print("\nSeasonality analysis completed for", len(seasonality_df), "restaurants")
print("\nSample of seasonality results:")
print(seasonality_df[['Restaurant_ID', 'Overall_Monthly_Avg', 'CV', 'Months_of_Data']].head(10))

Number of restaurants with at least 6 months of data: 97

Seasonality analysis completed for 96 restaurants

Sample of seasonality results:
                   Restaurant_ID  Overall_Monthly_Avg        CV  \
0              Andrea-Antisystem           399.012222  0.256033   
1  Assaggini-Wine-Bar-Mornington          1519.688235  0.319593   
2          Avoja-Pizzeria-sydney          1055.135714  0.707345   
3            Baby-Eggs-Melbourne            29.965714  0.709527   
4                  Bancho-Sydney           810.862222  0.526835   
5           Bang-Bang-Mordialloc           335.166667  0.607463   
6                Bar-Nina-Sydney          3999.170000  0.249562   
7            Beachcomber-StKilda           198.678750  0.287557   
8                   Bella-Darwin          1881.471000  0.554417   
9                Benzina-Cantina           527.943333  0.807312   

   Months_of_Data  
0              18  
1              17  
2               7  
3               7  
4               9  
5 

**Here, I have categorised restaurants into seasonality scores based on the coefficient of variation (CV). It filters out invalid CV values, calculates quintiles to establish thresholds, and assigns scores from 1 (least seasonal) to 5 (most seasonal) based on these thresholds. Additionally, it provides distribution summaries and examples for each score category to analyse the seasonality variability across restaurants.**

In [6]:
valid_seasonality_df = seasonality_df[seasonality_df['CV'].notna() & np.isfinite(seasonality_df['CV'])].copy()

print(f"Valid restaurants for scoring: {len(valid_seasonality_df)}")
print("\nCV distribution:")
print(valid_seasonality_df['CV'].describe())

cv_quintiles = valid_seasonality_df['CV'].quantile([0.2, 0.4, 0.6, 0.8, 1.0])
print("\nCV Quintiles:")
print(cv_quintiles)

def assign_seasonality_score(cv):
    if cv <= cv_quintiles[0.2]:
        return 1  
    elif cv <= cv_quintiles[0.4]:
        return 2  
    elif cv <= cv_quintiles[0.6]:
        return 3  
    elif cv <= cv_quintiles[0.8]:
        return 4  
    else:
        return 5  

valid_seasonality_df['Seasonality_Score'] = valid_seasonality_df['CV'].apply(assign_seasonality_score)

print("\nSeasonality Score Distribution:")
print(valid_seasonality_df['Seasonality_Score'].value_counts().sort_index())

print("\nExamples by Seasonality Score:")
for score in range(1, 6):
    lower_bound = cv_quintiles.iloc[score - 1] if score > 1 else valid_seasonality_df['CV'].min()
    upper_bound = cv_quintiles.iloc[score - 1] if score < 5 else valid_seasonality_df['CV'].max()
    
    examples = valid_seasonality_df[valid_seasonality_df['Seasonality_Score'] == score][['Restaurant_ID', 'CV', 'Overall_Monthly_Avg']].head(3)
    
    print(f"\nScore {score} (CV range: {lower_bound:.3f} - {upper_bound:.3f}):")
    print(examples)

Valid restaurants for scoring: 96

CV distribution:
count    96.000000
mean      0.576110
std       0.301836
min       0.118967
25%       0.358202
50%       0.515399
75%       0.744966
max       1.801033
Name: CV, dtype: float64

CV Quintiles:
0.2    0.320214
0.4    0.435755
0.6    0.597252
0.8    0.803920
1.0    1.801033
Name: CV, dtype: float64

Seasonality Score Distribution:
Seasonality_Score
1    20
2    19
3    19
4    19
5    19
Name: count, dtype: int64

Examples by Seasonality Score:

Score 1 (CV range: 0.119 - 0.320):
                   Restaurant_ID        CV  Overall_Monthly_Avg
0              Andrea-Antisystem  0.256033           399.012222
1  Assaggini-Wine-Bar-Mornington  0.319593          1519.688235
6                Bar-Nina-Sydney  0.249562          3999.170000

Score 2 (CV range: 0.436 - 0.436):
            Restaurant_ID        CV  Overall_Monthly_Avg
10  Blackbird-Cafe-Sydney  0.419253          7454.011250
11               Bottarga  0.377184          1054.595333
13 

**I am analysing the seasonality scores of restaurants by filtering out invalid CV values, calculating quintiles to categorise the CV into different seasonality levels, assigning scores based on these categories, and then examining the distribution and examples of each score category.**

In [7]:
valid_seasonality_df = seasonality_df[seasonality_df['CV'].notna() & np.isfinite(seasonality_df['CV'])].copy()

print(f"Valid restaurants for scoring: {len(valid_seasonality_df)}")
print("\nCV distribution:")
print(valid_seasonality_df['CV'].describe())

cv_quintiles = valid_seasonality_df['CV'].quantile([0.2, 0.4, 0.6, 0.8])
print("\nCV Quintiles:")
print(cv_quintiles)

def assign_seasonality_score(cv):
    if cv <= cv_quintiles[0.2]:
        return 1  
    elif cv <= cv_quintiles[0.4]:
        return 2  
    elif cv <= cv_quintiles[0.6]:
        return 3  
    elif cv <= cv_quintiles[0.8]:
        return 4  
    else:
        return 5  

valid_seasonality_df['Seasonality_Score'] = valid_seasonality_df['CV'].apply(assign_seasonality_score)

print("\nSeasonality Score Distribution:")
print(valid_seasonality_df['Seasonality_Score'].value_counts().sort_index())

print("\nExamples by Seasonality Score:")
for score in range(1, 6):
    examples = valid_seasonality_df[valid_seasonality_df['Seasonality_Score'] == score][['Restaurant_ID', 'CV', 'Overall_Monthly_Avg']].head(3)
    print(f"\nScore {score}:")
    print(examples)
    
    score_data = valid_seasonality_df[valid_seasonality_df['Seasonality_Score'] == score]
    if len(score_data) > 0:
        print(f"CV range: {score_data['CV'].min():.3f} - {score_data['CV'].max():.3f}")

Valid restaurants for scoring: 96

CV distribution:
count    96.000000
mean      0.576110
std       0.301836
min       0.118967
25%       0.358202
50%       0.515399
75%       0.744966
max       1.801033
Name: CV, dtype: float64

CV Quintiles:
0.2    0.320214
0.4    0.435755
0.6    0.597252
0.8    0.803920
Name: CV, dtype: float64

Seasonality Score Distribution:
Seasonality_Score
1    20
2    19
3    19
4    19
5    19
Name: count, dtype: int64

Examples by Seasonality Score:

Score 1:
                   Restaurant_ID        CV  Overall_Monthly_Avg
0              Andrea-Antisystem  0.256033           399.012222
1  Assaggini-Wine-Bar-Mornington  0.319593          1519.688235
6                Bar-Nina-Sydney  0.249562          3999.170000
CV range: 0.119 - 0.320

Score 2:
            Restaurant_ID        CV  Overall_Monthly_Avg
10  Blackbird-Cafe-Sydney  0.419253          7454.011250
11               Bottarga  0.377184          1054.595333
13          Cardea-Sydney  0.419701          16

**Running code to generate sales forecasts for the selected restaurant by analysing historical data, creating future periods, and applying a linear regression model to produce estimates for the next six months.**

In [8]:
top_restaurants = valid_seasonality_df.nlargest(10, 'Months_of_Data')
print("Top 10 restaurants by months of data:")
print(top_restaurants[['Restaurant_ID', 'Months_of_Data', 'CV', 'Seasonality_Score', 'Overall_Monthly_Avg']])

forecast_restaurant = 'saga-bar-sydney'
forecast_data = qualified_data[qualified_data['Restaurant ID'] == forecast_restaurant].copy()
forecast_data = forecast_data.sort_values('YearMonth')

print(f"\nForecast data for {forecast_restaurant}:")
print(f"Date range: {forecast_data['YearMonth'].min()} to {forecast_data['YearMonth'].max()}")
print(f"Number of months: {len(forecast_data)}")

forecast_data['Period'] = range(1, len(forecast_data) + 1)
forecast_data['YearMonth_str'] = forecast_data['YearMonth'].astype(str)

print("\nHistorical data:")
print(forecast_data[['YearMonth_str', 'Revenue_Sum', 'Period']].head(10))

last_period = forecast_data['Period'].max()
last_date = forecast_data['YearMonth'].max()

future_periods = []
for i in range(1, 7):
    future_date = last_date + i
    future_periods.append({
        'Period': last_period + i,
        'YearMonth': future_date,
        'YearMonth_str': str(future_date),
        'Revenue_Sum': None,  # To be forecasted
        'Type': 'Forecast'
    })

# Add type column to historical data
forecast_data['Type'] = 'Historical'

print(f"\nFuture periods to forecast:")
for period in future_periods:
    print(f"Period {period['Period']}: {period['YearMonth_str']}")

# Simple linear trend forecast as backup (Excel will use FORECAST.ETS)
from sklearn.linear_model import LinearRegression
import numpy as np

X = forecast_data['Period'].values.reshape(-1, 1)
y = forecast_data['Revenue_Sum'].values

model = LinearRegression()
model.fit(X, y)

simple_forecasts = []
for period in future_periods:
    pred = model.predict([[period['Period']]])[0]
    simple_forecasts.append(max(0, pred))  # Ensure non-negative

print(f"\nSimple linear forecast for next 6 months:")
for i, period in enumerate(future_periods):
    print(f"{period['YearMonth_str']}: ${simple_forecasts[i]:,.2f}")

from sklearn.metrics import mean_absolute_error, mean_squared_error

train_pred = model.predict(X)
mae = mean_absolute_error(y, train_pred)
rmse = np.sqrt(mean_squared_error(y, train_pred))

print(f"\nModel performance on historical data:")
print(f"MAE: ${mae:,.2f}")
print(f"RMSE: ${rmse:,.2f}")
print(f"Mean Revenue: ${np.mean(y):,.2f}")
print(f"MAPE: {(mae/np.mean(y))*100:.1f}%")

Top 10 restaurants by months of data:
                   Restaurant_ID  Months_of_Data        CV  Seasonality_Score  \
95               saga-bar-sydney              23  0.157735                  1   
26            Entrata-restaurant              21  0.765169                  4   
30          Imaccheroni-catering              20  0.852271                  5   
56            Onda-Bar-Melbourne              20  0.358462                  2   
19              Daydreamers-Cafe              19  0.320214                  1   
27            Fitzrovia-St-Kilda              19  0.516632                  3   
45  Martini-and-Co-Plenty-Valley              19  0.261711                  1   
51           Mercetta-Mornington              19  0.178133                  1   
0              Andrea-Antisystem              18  0.256033                  1   
93                 iceberg-bondi              18  0.418215                  2   

    Overall_Monthly_Avg  
95          2484.310870  
26           639.6

**Generating the output and summarising it into an Excel file to show all the data cleaning and structuring along with Forecast output for the next 6 months for SAGA-BAR-SYDNEY**

In [9]:
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils.dataframe import dataframe_to_rows
import pandas as pd

wb = Workbook()

wb.remove(wb.active)

exec_sheet = wb.create_sheet("Executive Summary")
exec_sheet.append(["QLUB RESTAURANT SEASONALITY ANALYSIS"])
exec_sheet.append(["Analysis Date:", "July 18, 2025"])
exec_sheet.append([])
exec_sheet.append(["KEY FINDINGS:"])
exec_sheet.append(["• Total Restaurants Analyzed:", len(valid_seasonality_df)])
exec_sheet.append(["• Data Period:", "Jan 2023 - Dec 2024"])
exec_sheet.append(["• Average Monthly Revenue:", f"${valid_seasonality_df['Overall_Monthly_Avg'].mean():,.2f}"])
exec_sheet.append([])
exec_sheet.append(["SEASONALITY SCORE DISTRIBUTION:"])
for score in range(1, 6):
    count = len(valid_seasonality_df[valid_seasonality_df['Seasonality_Score'] == score])
    exec_sheet.append([f"Score {score}:", f"{count} restaurants ({count/len(valid_seasonality_df)*100:.1f}%)"])

scores_sheet = wb.create_sheet("Restaurant Seasonality Scores")
scores_data = valid_seasonality_df[['Restaurant_ID', 'Seasonality_Score', 'CV', 'Overall_Monthly_Avg', 'Months_of_Data']].sort_values('Seasonality_Score')

scores_sheet.append(['Restaurant ID', 'Seasonality Score', 'Coefficient of Variation', 'Avg Monthly Revenue', 'Months of Data'])

for index, row in scores_data.iterrows():
    scores_sheet.append([
        row['Restaurant_ID'],
        row['Seasonality_Score'],
        round(row['CV'], 3),
        round(row['Overall_Monthly_Avg'], 2),
        row['Months_of_Data']
    ])

detail_sheet = wb.create_sheet("Detailed Seasonality Analysis")

headers = ['Restaurant_ID', 'Seasonality_Score', 'CV', 'Overall_Monthly_Avg']
for month in range(1, 13):
    headers.append(f'Jan Feb Mar Apr May Jun Jul Aug Sep Oct Nov Dec'.split()[month-1])
    
detail_sheet.append(headers)

for index, row in valid_seasonality_df.iterrows():
    row_data = [
        row['Restaurant_ID'],
        row['Seasonality_Score'],
        round(row['CV'], 3),
        round(row['Overall_Monthly_Avg'], 2)
    ]
    
    for month in range(1, 13):
        col_name = f'Month_{month}_Index'
        if col_name in row:
            value = row[col_name]
            if pd.notna(value):
                row_data.append(round(value, 3))
            else:
                row_data.append('')
        else:
            row_data.append('')
    
    detail_sheet.append(row_data)

forecast_sheet = wb.create_sheet("Forecast Model")
forecast_sheet.append(["FORECAST MODEL FOR SAGA-BAR-SYDNEY"])
forecast_sheet.append([])
forecast_sheet.append(["Historical Data:"])
forecast_sheet.append(["Month", "Revenue", "Period"])

for index, row in forecast_data.iterrows():
    forecast_sheet.append([
        row['YearMonth_str'],
        round(row['Revenue_Sum'], 2),
        row['Period']
    ])

forecast_sheet.append([])
forecast_sheet.append(["FORECAST (Next 6 Months):"])
forecast_sheet.append(["Month", "Forecasted Revenue", "Method"])

for i, period in enumerate(future_periods):
    forecast_sheet.append([
        period['YearMonth_str'],
        round(simple_forecasts[i], 2),
        "Linear Regression"
    ])

forecast_sheet.append([])
forecast_sheet.append(["MODEL PERFORMANCE:"])
forecast_sheet.append(["MAE:", f"${mae:,.2f}"])
forecast_sheet.append(["RMSE:", f"${rmse:,.2f}"])
forecast_sheet.append(["MAPE:", f"{(mae/np.mean(y))*100:.1f}%"])

rec_sheet = wb.create_sheet("Recommendations")
rec_sheet.append(["OPERATIONAL RECOMMENDATIONS BY SEASONALITY SCORE"])
rec_sheet.append([])
rec_sheet.append(["SCORE 1 - LOW SEASONALITY (CV < 0.32):"])
rec_sheet.append(["• Consistent revenue patterns - ideal for predictable operations"])
rec_sheet.append(["• Focus on efficiency optimizations and cost management"])
rec_sheet.append(["• Examples: saga-bar-sydney, Daydreamers-Cafe"])
rec_sheet.append([])
rec_sheet.append(["SCORE 2 - LOW-MEDIUM SEASONALITY (CV 0.33-0.44):"])
rec_sheet.append(["• Moderate revenue variations"])
rec_sheet.append(["• Implement flexible staffing strategies"])
rec_sheet.append(["• Examples: Onda-Bar-Melbourne, iceberg-bondi"])
rec_sheet.append([])
rec_sheet.append(["SCORE 3 - MEDIUM SEASONALITY (CV 0.45-0.60):"])
rec_sheet.append(["• Noticeable seasonal patterns"])
rec_sheet.append(["• Develop seasonal menu offerings"])
rec_sheet.append(["• Examples: Fitzrovia-St-Kilda, Bancho-Sydney"])
rec_sheet.append([])
rec_sheet.append(["SCORE 4 - HIGH-MEDIUM SEASONALITY (CV 0.61-0.80):"])
rec_sheet.append(["• Strong seasonal variations"])
rec_sheet.append(["• Implement dynamic pricing strategies"])
rec_sheet.append(["• Examples: Entrata-restaurant, Avoja-Pizzeria-sydney"])
rec_sheet.append([])
rec_sheet.append(["SCORE 5 - HIGH SEASONALITY (CV > 0.80):"])
rec_sheet.append(["• Extreme seasonal variations"])
rec_sheet.append(["• Focus on peak season maximization"])
rec_sheet.append(["• Consider alternative revenue streams in low periods"])
rec_sheet.append(["• Examples: Imaccheroni-catering, Benzina-Cantina"])

wb.save("Restaurant_Seasonality_Analysis.xlsx")

print("\n" + "="*50)
print("SEASONALITY ANALYSIS SUMMARY")
print("="*50)
print(f"Total Restaurants Analyzed: {len(valid_seasonality_df)}")
print(f"Date Range: {df_clean['Date Level [Finance]'].min().strftime('%Y-%m-%d')} to {df_clean['Date Level [Finance]'].max().strftime('%Y-%m-%d')}")
print(f"Total Records: {len(df_clean):,}")
print(f"Average Monthly Revenue: ${valid_seasonality_df['Overall_Monthly_Avg'].mean():,.2f}")
print(f"Average CV: {valid_seasonality_df['CV'].mean():.3f}")

print("\nSeasonality Score Distribution:")
for score in range(1, 6):
    count = len(valid_seasonality_df[valid_seasonality_df['Seasonality_Score'] == score])
    percentage = count/len(valid_seasonality_df)*100
    print(f"Score {score}: {count} restaurants ({percentage:.1f}%)")

print("\nTop 5 Most Seasonal Restaurants (Score 5):")
top_seasonal = valid_seasonality_df[valid_seasonality_df['Seasonality_Score'] == 5].nlargest(5, 'CV')
for index, row in top_seasonal.iterrows():
    print(f"• {row['Restaurant_ID']} (CV: {row['CV']:.3f})")

print("\nTop 5 Most Consistent Restaurants (Score 1):")
top_consistent = valid_seasonality_df[valid_seasonality_df['Seasonality_Score'] == 1].nsmallest(5, 'CV')
for index, row in top_consistent.iterrows():
    print(f"• {row['Restaurant_ID']} (CV: {row['CV']:.3f})")

print("\nForecast for saga-bar-sydney (next 6 months):")
for i, period in enumerate(future_periods):
    print(f"• {period['YearMonth_str']}: ${simple_forecasts[i]:,.2f}")

print(f"\nModel Accuracy: {(mae/np.mean(y))*100:.1f}% MAPE")


SEASONALITY ANALYSIS SUMMARY
Total Restaurants Analyzed: 96
Date Range: 2023-01-05 to 2024-12-31
Total Records: 24,852
Average Monthly Revenue: $2,226.55
Average CV: 0.576

Seasonality Score Distribution:
Score 1: 20 restaurants (20.8%)
Score 2: 19 restaurants (19.8%)
Score 3: 19 restaurants (19.8%)
Score 4: 19 restaurants (19.8%)
Score 5: 19 restaurants (19.8%)

Top 5 Most Seasonal Restaurants (Score 5):
• PAYMENT-LINKS-Stokehouse-pasta (CV: 1.801)
• Emu-Bottom-Homestead (CV: 1.370)
• Stray-Neighbour (CV: 1.266)
• VPOS-Lumi-Sydney (CV: 1.194)
• House-of-Fides-Melbourne (CV: 1.165)

Top 5 Most Consistent Restaurants (Score 1):
• LuMi-Dining-Sydney (CV: 0.119)
• saga-bar-sydney (CV: 0.158)
• INDU-Dining-sydney (CV: 0.174)
• Mercetta-Mornington (CV: 0.178)
• Mr-Mrs-P (CV: 0.192)

Forecast for saga-bar-sydney (next 6 months):
• 2025-01: $2,431.45
• 2025-02: $2,427.05
• 2025-03: $2,422.64
• 2025-04: $2,418.24
• 2025-05: $2,413.83
• 2025-06: $2,409.43

Model Accuracy: 12.5% MAPE
